In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os.path
import tensorflow as tf
import json
from sklearn.model_selection import train_test_split
from  sklearn.manifold import TSNE as TSNE
from sklearn.svm import SVC as SVC
from sklearn.decomposition import PCA as PCA

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [3]:
import os
# os.chdir("/content/drive/MyDrive/AM216/Mini Project")
os.chdir(".")

local_data = np.loadtxt('local.csv')
global_data = np.loadtxt('global.csv')
labels = np.loadtxt('labels.csv')

local_train, local_test, y_train, y_test = train_test_split(local_data, labels, test_size=0.2, random_state=1)

global_train, global_test, y_train2, y_test2 = train_test_split(global_data, labels, test_size=0.2, random_state=1)

### Generate the mini labeled light intensity plots to put inside the low dim tsne space
### Computationally expensive so only do once

# plt.clf
# for pp in range(1640,3148):
# # for pp in range(8):
# #     print(pp)
#     ax = plt.axes()
#     plt.plot(local_test[pp,:],color = "black")
#     plt.yticks([])
#     plt.xticks([])
#     if y_test[pp] == 0:
#         ax.set_facecolor("red")
#     if y_test[pp] == 1:
#         ax.set_facecolor("blue")
#     plt.savefig("plots/lc" + str(pp) + ".png" )
# #     plt.show()
#     plt.clf
   

In [7]:
model = tf.keras.models.load_model("conv_model")

X = model.layers[-2].output
model2 = tf.keras.Model(inputs = (local_input,global_input),outputs = X)
model2.summary()

X = model2.predict((local_test,global_test))
y = y_test

X = local_test[1:1000]
y = y_test[1:1000]

# Scale and visualize the embedding vectors
def plot_embedding(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure(figsize = (18,12))
    ax = plt.subplot(111)
#     for i in range(X.shape[0]):
#         plt.text(X[i, 0], X[i, 1], str(y[i]),
#                  color=plt.cm.Set1(y[i]),
#                  fontdict={'weight': 'bold', 'size': 9})

    if hasattr(offsetbox, 'AnnotationBbox'):
#         with open('Exoplanets/PC/0.json', 'r') as fp:
#             exo_dictionary = json.load(fp)
#         with "plots/lc" + str(pp) + ".png" as file:
        # only print thumbnails with matplotlib > 1.0
        shown_images = np.array([[1., 1.]])  # just something big
        for i in range(X.shape[0]):
            dist = np.sum((X[i] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-3:
                # don't show points that are too close
                continue
            shown_images = np.r_[shown_images, [X[i]]]
            arr_img = plt.imread("plots/lc" + str(i) + ".png")
            
            imagebox = offsetbox.AnnotationBbox(
#                 offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r)
                offsetbox.OffsetImage( arr_img , cmap=plt.cm.gray_r,zoom = .05)
                , X[i]
                , xybox= 1*X[i]
                , xycoords = 'data'
            )
            ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

### Run Tsne embedding and plot 
plt.clf
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
t0 = time()
X_tsne = tsne.fit_transform(X)

plot_embedding(X_tsne
#                "t-SNE embedding of the digits (time %.2fs)" %
#                (time() - t0)
              )
plt.show()

OSError: Unable to open file (file read failed: time = Tue Apr 19 10:32:04 2022
, filename = 'conv_model', file descriptor = 58, errno = 21, error message = 'Is a directory', buf = 0x7ffee650ce38, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)